# Public Transit Play II 

1. Find "The shortest route to iterate all subway lines in ShangHai."
2. Find "The Longest route without same edge in subway network of ShangHai & Beijing"

Answers of 1. can be found on Zhihu: https://www.zhihu.com/question/629922852/answer/54798222956


In [70]:
import gurobipy as grb


In [ ]:
file_path = "./data/PublicTransit/SH.txt"  # 替换为你的实际文件路径
# file_path = "./data/PublicTransit/BJ.txt"  # 替换为你的实际文件路径
# file_path = "./data/PublicTransit/NJ.txt"
city = "SH" # 设置城市

In [102]:
# import gurobipy as grb
# def read_metro_data(file_path):
#     from collections import defaultdict
#     # 邻接表表示的地铁网络图
#     metro_network = defaultdict(dict)
#     line_routes = dict()
#     station_name_to_idx = dict()
#     idx_to_station_name = dict()
#     cnt_idx = 1
#     # transship = dict()
#     with open(file_path, 'r', encoding='utf-8') as file:
#         current_line = None
#         for line in file:
#             line = line.strip()
#             if line.startswith("Line"):
#                 # 读取线路名
#                 current_line = line
#                 line_routes[line] = list()
#             elif line:
#                 # 读取站点和距离信息
#                 station1, station2, distance = line.split(",")
#                 if station1 not in station_name_to_idx:
#                     station_name_to_idx[station1] = cnt_idx
#                     idx_to_station_name[cnt_idx] = station1
#                     cnt_idx += 1
#                 if station2 not in station_name_to_idx:
#                     station_name_to_idx[station2] = cnt_idx
#                     idx_to_station_name[cnt_idx] = station2
#                     cnt_idx += 1
#                 distance = int(distance)  # 转换距离为整数
#                 metro_network[station1][station2] = distance
#                 metro_network[station2][station1] = distance
#                 # metro_network[station1][station2] = 1
#                 # metro_network[station2][station1] = 1
#                 line_routes[current_line].append(f"{station1},{station2}")
                

#     return metro_network, line_routes, station_name_to_idx, idx_to_station_name

# # 示例调用
# metro_network, line_routes, station_name_to_idx, idx_to_station_name = read_metro_data(file_path)

# # 打印部分结果查看结构
# for station, connections in list(metro_network.items())[:5]:
#     print(f"Station: {station}, Connections: {connections}")

def read_metro_data(file_path):
    from collections import defaultdict
    # 邻接表表示的地铁网络图
    metro_network = defaultdict(dict)
    line_routes = dict()
    station_name_to_idx = dict()
    idx_to_station_name = dict()
    trans_stations = dict() # 记录换乘站
    cnt_idx = 1
    # transship = dict()
    with open(file_path, 'r', encoding='utf-8') as file:
        current_line = None
        for line in file:
            line = line.strip()
            if line.startswith("Line"):
                # 读取线路名
                current_line = line
                line_routes[line] = list()
            elif line:
                # 读取站点和距离信息
                station1, station2, distance = line.split(",")
                if station1 not in trans_stations:
                    trans_stations[station1] = set([current_line])
                else:
                    trans_stations[station1].add(current_line)
                if station2 not in trans_stations: 
                    trans_stations[station2] = set([current_line])
                else:
                    trans_stations[station2].add(current_line)
                if station1 not in station_name_to_idx:
                    station_name_to_idx[station1] = cnt_idx
                    idx_to_station_name[cnt_idx] = station1
                    cnt_idx += 1
                if station2 not in station_name_to_idx:
                    station_name_to_idx[station2] = cnt_idx
                    idx_to_station_name[cnt_idx] = station2
                    cnt_idx += 1
                distance = int(distance)  # 转换距离为整数
                metro_network[f"{station1},{current_line}"][f"{station2},{current_line}"] = distance
                metro_network[f"{station2},{current_line}"][f"{station1},{current_line}"] = distance
                # metro_network[station1][station2] = 1
                # metro_network[station2][station1] = 1
                line_routes[current_line].append(f"{station1},{station2}")
                

    return metro_network, line_routes, station_name_to_idx, idx_to_station_name, trans_stations

def reconstruct_index(line_routes):
    idx = 0
    aoi_set = set()
    for k, v in line_routes.items():
        for v_ in v:
            o_, d_ = v_.split(",")
            if f"{o_},{k}" not in aoi_set:
                aoi_set.add(f"{o_},{k}")
            if f"{d_},{k}" not in aoi_set:
                aoi_set.add(f"{d_},{k}")
    aoi_set = list(aoi_set)
    station_name_to_idx = dict()
    idx_to_station_name = dict()
    for idx, aoi_name in enumerate(aoi_set):
        station_name_to_idx[aoi_name] = idx + 1
        idx_to_station_name[idx + 1] = aoi_name
    return aoi_set, station_name_to_idx, idx_to_station_name


metro_network, line_routes, _ , _, trans_stations = read_metro_data(file_path)
station_set, station_name_to_idx, idx_to_station_name = reconstruct_index(line_routes)

# 打印部分结果查看结构
for station, connections in list(metro_network.items())[:5]:
    print(f"Station: {station}, Connections: {connections}")
for s_name, s_idx in list(station_name_to_idx.items())[:5]:
    print(f"Station: {s_name}, Index: {s_idx}")
# for k, v in trans_stations.items():
#     if len(v) > 1:
#         print(k, v)


Station: 莘庄,Line 1, Connections: {'外环路,Line 1': 1303}
Station: 外环路,Line 1, Connections: {'莘庄,Line 1': 1303, '莲花路,Line 1': 1459}
Station: 莲花路,Line 1, Connections: {'外环路,Line 1': 1459, '锦江乐园,Line 1': 1633}
Station: 锦江乐园,Line 1, Connections: {'莲花路,Line 1': 1633, '上海南站,Line 1': 2089}
Station: 上海南站,Line 1, Connections: {'锦江乐园,Line 1': 2089, '漕宝路,Line 1': 1661}
Station: 上南路,Line 6, Index: 1
Station: 新场,Line 16, Index: 2
Station: 陕西南路,Line 12, Index: 3
Station: 港城路,Line 10, Index: 4
Station: 上海南站,Line 3, Index: 5


### Gurobi Version (Fast!): Q1

In [69]:
ptn = grb.Model("PTN")
idx_to_station_name[0] = "Dummy"
station_name_to_idx["Dummy"] = 0
x = {}
y = {}
n = {}
N = len(station_name_to_idx.keys())
M = 2000
out_flow = dict()

for i in range(1, N):
    x[0, i] = ptn.addVar(vtype=grb.GRB.BINARY,obj = 0, name=f"x_Dummy_{idx_to_station_name[i]}")
    x[i, 0] = ptn.addVar(vtype=grb.GRB.BINARY,obj = 0, name=f"x_{idx_to_station_name[i]}_Dummy")
    y[i] = ptn.addVar(vtype=grb.GRB.BINARY,obj = 0, name=f"y_{idx_to_station_name[i]}")
    n[i] = ptn.addVar(vtype=grb.GRB.INTEGER,obj = 0, lb = 0, name=f"n_{idx_to_station_name[i]}")
    out_flow[i] = []

for station, connections in list(metro_network.items()):
    idx_1 = station_name_to_idx[station]
    for next_station, dist in connections.items():
        idx_2 = station_name_to_idx[next_station]
        x[idx_1, idx_2] = ptn.addVar(vtype=grb.GRB.BINARY,obj = dist, name=f"x_{station}_{next_station}")
        # x[idx_1, idx_2] = ptn.addVar(vtype=grb.GRB.BINARY,obj = 1, name=f"x_{station}_{next_station}")
        out_flow[idx_1].append(idx_2)
# 针对换乘的情况, 给出带转运的解


# 流出的
for k, v in out_flow.items():

    v_new = v + [0]
    ptn.addConstr(grb.quicksum(x[k, j] for j in v_new) == y[k], name=f"out_flow_{idx_to_station_name[k]}")
    ptn.addConstr(grb.quicksum(x[i, k] for i in v_new) == y[k], name=f"in_flow_{idx_to_station_name[k]}")


ptn.addConstr(grb.quicksum(x[i, 0] for i in range(1, N)) == 1, name=f"Dummy_balance1")
ptn.addConstr(grb.quicksum(x[0, i] for i in range(1, N)) == 1, name=f"Dummy_balance2")

for i in range(1, N):
    ptn.addConstr(n[i] <= M * y[i], name = f"N_{i}")

for station, connections in list(metro_network.items()):
    idx_1 = station_name_to_idx[station]
    for next_station, dist in connections.items():
        idx_2 = station_name_to_idx[next_station]
        ptn.addConstr(n[idx_2] - n[idx_1] >= 1 + M * (x[idx_1, idx_2] - 1), name = f"n_{idx_1}_{idx_2}")

# 限制每条线都至少要走一次
for k, edges in line_routes.items():
    left_edges = []
    for edge in edges:
        o_, d_ = edge.split(",")
        o_idx, d_idx = station_name_to_idx[o_], station_name_to_idx[d_]
        left_edges.append([o_idx, d_idx])
        left_edges.append([d_idx, o_idx])
    ptn.addConstr(grb.quicksum(x[i, j] for i, j in left_edges) >= 1, name = f"line_{k}")

# 处理 Line 3 + Line 4 的情况(上海地铁)
edges_3, edges_4 = line_routes["Line 3"], line_routes["Line 4"]
new_edges = list(set(edges_3 + edges_4))
left_edges = []
for edge in new_edges:
    o_, d_ = edge.split(",")
    o_idx, d_idx = station_name_to_idx[o_], station_name_to_idx[d_]
    left_edges.append([o_idx, d_idx])
    left_edges.append([d_idx, o_idx])
ptn.addConstr(grb.quicksum(x[i, j] for i, j in left_edges) >= 2, name = f"spec_line_3/4")

edges_2, edges_10 = line_routes["Line 2"], line_routes["Line 10"]
new_edges = list(set(edges_2 + edges_10))
left_edges = []
for edge in new_edges:
    o_, d_ = edge.split(",")
    o_idx, d_idx = station_name_to_idx[o_], station_name_to_idx[d_]
    left_edges.append([o_idx, d_idx])
    left_edges.append([d_idx, o_idx])
ptn.addConstr(grb.quicksum(x[i, j] for i, j in left_edges) >= 2, name = f"spec_line_10/2")


ptn.modelSense = grb.GRB.MINIMIZE
# ptn.modelSense = grb.GRB.MAXIMIZE
result_routes = []
ptn.optimize()
if (ptn.status == grb.GRB.status.OPTIMAL):
    solType = 'IP_Optimal'
    ofv = ptn.getObjective().getValue()
    print(f"BEST VALUE:{ofv}")
    for i, j in x:
        if (x[i, j].x > 0.5):
            route = f"{idx_to_station_name[i]},{idx_to_station_name[j]}"
            result_routes.append(route)
            print(route)

    gap = 0
    lb = ofv
    ub = ofv
    runtime = ptn.Runtime

Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (mac64[arm] - Darwin 23.5.0 23F79)

CPU model: Apple M3 Pro
Thread count: 11 physical cores, 11 logical processors, using up to 11 threads

Optimize a model with 2208 rows, 2592 columns and 9296 nonzeros
Model fingerprint: 0x701983a3
Variable types: 0 continuous, 2592 integer (2186 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [6e+02, 1e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+03]
Presolve time: 0.01s
Presolved: 2208 rows, 2592 columns, 9296 nonzeros
Variable types: 0 continuous, 2592 integer (2186 binary)

Root relaxation: objective 1.541800e+04, 211 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 15418.0000    0   56          - 15418.0000      -     -    0s
     0     0 16532.0000    0   81          - 16532.0000 

## Gurobi for Q2

In [ ]:
ptn = grb.Model("PTN")
idx_to_station_name[0] = "Dummy"
station_name_to_idx["Dummy"] = 0
x = {}
y = {}
n = {}
N = len(station_name_to_idx.keys())
M = 2000
out_flow = dict()

for i in range(1, N):
    x[0, i] = ptn.addVar(vtype=grb.GRB.BINARY,obj = 0, name=f"x_Dummy_{idx_to_station_name[i]}")
    x[i, 0] = ptn.addVar(vtype=grb.GRB.BINARY,obj = 0, name=f"x_{idx_to_station_name[i]}_Dummy")
    y[i] = ptn.addVar(vtype=grb.GRB.BINARY,obj = 0, name=f"y_{idx_to_station_name[i]}")
    n[i] = ptn.addVar(vtype=grb.GRB.INTEGER,obj = 0, lb = 0, name=f"n_{idx_to_station_name[i]}")
    out_flow[i] = []

for station, connections in list(metro_network.items()):
    idx_1 = station_name_to_idx[station]
    for next_station, dist in connections.items():
        idx_2 = station_name_to_idx[next_station]
        # x[idx_1, idx_2] = ptn.addVar(vtype=grb.GRB.BINARY,obj = dist, name=f"x_{station}_{next_station}")
        x[idx_1, idx_2] = ptn.addVar(vtype=grb.GRB.BINARY,obj = 1, name=f"x_{station}_{next_station}")
        out_flow[idx_1].append(idx_2)
# 针对换乘的情况, 给出带转运的解


# 流出的
for k, v in out_flow.items():

    v_new = v + [0]
    ptn.addConstr(grb.quicksum(x[k, j] for j in v_new) == y[k], name=f"out_flow_{idx_to_station_name[k]}")
    ptn.addConstr(grb.quicksum(x[i, k] for i in v_new) == y[k], name=f"in_flow_{idx_to_station_name[k]}")


ptn.addConstr(grb.quicksum(x[i, 0] for i in range(1, N)) == 1, name=f"Dummy_balance1")
ptn.addConstr(grb.quicksum(x[0, i] for i in range(1, N)) == 1, name=f"Dummy_balance2")

for i in range(1, N):
    ptn.addConstr(n[i] <= M * y[i], name = f"N_{i}")

for station, connections in list(metro_network.items()):
    idx_1 = station_name_to_idx[station]
    for next_station, dist in connections.items():
        idx_2 = station_name_to_idx[next_station]
        ptn.addConstr(n[idx_2] - n[idx_1] >= 1 + M * (x[idx_1, idx_2] - 1), name = f"n_{idx_1}_{idx_2}")

# 限制每条线都至少要走一次
for k, edges in line_routes.items():
    left_edges = []
    for edge in edges:
        o_, d_ = edge.split(",")
        o_idx, d_idx = station_name_to_idx[o_], station_name_to_idx[d_]
        left_edges.append([o_idx, d_idx])
        left_edges.append([d_idx, o_idx])
    ptn.addConstr(grb.quicksum(x[i, j] for i, j in left_edges) >= 1, name = f"line_{k}")

# # 处理 Line 3 + Line 4 的情况(上海地铁)
# edges_3, edges_4 = line_routes["Line 3"], line_routes["Line 4"]
# new_edges = list(set(edges_3 + edges_4))
# left_edges = []
# for edge in new_edges:
#     o_, d_ = edge.split(",")
#     o_idx, d_idx = station_name_to_idx[o_], station_name_to_idx[d_]
#     left_edges.append([o_idx, d_idx])
#     left_edges.append([d_idx, o_idx])
# ptn.addConstr(grb.quicksum(x[i, j] for i, j in left_edges) >= 2, name = f"spec_line_3/4")

# edges_2, edges_10 = line_routes["Line 2"], line_routes["Line 10"]
# new_edges = list(set(edges_2 + edges_10))
# left_edges = []
# for edge in new_edges:
#     o_, d_ = edge.split(",")
#     o_idx, d_idx = station_name_to_idx[o_], station_name_to_idx[d_]
#     left_edges.append([o_idx, d_idx])
#     left_edges.append([d_idx, o_idx])
# ptn.addConstr(grb.quicksum(x[i, j] for i, j in left_edges) >= 2, name = f"spec_line_10/2")


# ptn.modelSense = grb.GRB.MINIMIZE
ptn.modelSense = grb.GRB.MINIMIZE
result_routes = []
ptn.optimize()
if (ptn.status == grb.GRB.status.OPTIMAL):
    solType = 'IP_Optimal'
    ofv = ptn.getObjective().getValue()
    print(f"BEST VALUE:{ofv}")
    for i, j in x:
        if (x[i, j].x > 0.5):
            route = f"{idx_to_station_name[i]},{idx_to_station_name[j]}"
            result_routes.append(route)
            print(route)

    gap = 0
    lb = ofv
    ub = ofv
    runtime = ptn.Runtime

Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (mac64[arm] - Darwin 23.5.0 23F79)

CPU model: Apple M3 Pro
Thread count: 11 physical cores, 11 logical processors, using up to 11 threads

Optimize a model with 2017 rows, 2362 columns and 8304 nonzeros
Model fingerprint: 0xa6673017
Variable types: 0 continuous, 2362 integer (1995 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+03]
Presolve time: 0.01s
Presolved: 2017 rows, 2362 columns, 8304 nonzeros
Variable types: 0 continuous, 2362 integer (1995 binary)

Root relaxation: objective 1.800000e+01, 475 iterations, 0.00 seconds (0.01 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   18.00000    0   56          -   18.00000      -     -    0s
     0     0   18.50000    0   73          -   18.50000 

## ortools version (Open source!): Q1

In [70]:
from ortools.sat.python import cp_model
import collections
# 2. 模型初始化 (OR-Tools)
# =================================
model = cp_model.CpModel()

# 添加虚拟节点 Dummy，与Gurobi代码逻辑一致
idx_to_station_name[0] = "Dummy"
station_name_to_idx["Dummy"] = 0

N = len(station_name_to_idx.keys())
M = 2000  # "Big M" 常数，与Gurobi代码一致

# 3. 变量定义 (OR-Tools)
# =================================
x = {}
y = {}
n = {}
out_flow = collections.defaultdict(list)

# 定义 y 和 n 变量
# 索引从1开始，因为虚拟节点0没有 y 和 n 变量
for i in range(1, N):
    y[i] = model.NewBoolVar(f"y_{idx_to_station_name[i]}")
    # n[i] 的上界可以是 N 或者 M，这里用 M 保持一致
    n[i] = model.NewIntVar(0, M, f"n_{idx_to_station_name[i]}")

# 定义 x 变量，并构建出度邻接表 out_flow
# 真实站点之间的边
for station, connections in metro_network.items():
    idx_1 = station_name_to_idx[station]
    for next_station, dist in connections.items():
        idx_2 = station_name_to_idx[next_station]
        # 确保只为每个无向边创建一次变量（尽管模型处理有向边）
        if (idx_1, idx_2) not in x:
            x[idx_1, idx_2] = model.NewBoolVar(f"x_{station}_{next_station}")
        out_flow[idx_1].append(idx_2)

# 与虚拟节点相关的边
for i in range(1, N):
    x[0, i] = model.NewBoolVar(f"x_Dummy_{idx_to_station_name[i]}")
    x[i, 0] = model.NewBoolVar(f"x_{idx_to_station_name[i]}_Dummy")

# 4. 约束定义 (OR-Tools)
# =================================

# 流量守恒约束
for k in range(1, N):
    # 构建进入和流出k的完整边集，包括与虚拟节点的连接
    out_neighbors = out_flow[k] + [0]
    in_neighbors = out_flow[k] + [0] # 对于无向图的对称创建，in-neighbors等同于out-neighbors
    
    # 出度约束
    model.Add(sum(x.get((k, j), 0) for j in out_neighbors) == y[k])
    # 入度约束
    model.Add(sum(x.get((i, k), 0) for i in in_neighbors) == y[k])

# 虚拟节点流量平衡约束
model.Add(sum(x[0, i] for i in range(1, N)) == 1)
model.Add(sum(x[i, 0] for i in range(1, N)) == 1)


# 子回路消除 (MTZ) 约束
for i in range(1, N):
    # Gurobi: ptn.addConstr(n[i] <= M * y[i])
    # OR-Tools等价实现: 如果 y[i]为0, 那么 n[i]必须为0
    model.Add(n[i] == 0).OnlyEnforceIf(y[i].Not())
    # 或者直接使用线性约束，与 Gurobi 完全一致
    # model.Add(n[i] <= M * y[i]) # 这句也可以，但上面的逻辑更紧凑

# Gurobi: ptn.addConstr(n[idx_2] - n[idx_1] >= 1 + M * (x[idx_1, idx_2] - 1))
# OR-Tools 等价实现: 如果 x[idx_1, idx_2] = 1, 则 n[idx_2] >= n[idx_1] + 1
for station, connections in metro_network.items():
    idx_1 = station_name_to_idx[station]
    for next_station, dist in connections.items():
        idx_2 = station_name_to_idx[next_station]
        # 添加两个方向的约束
        model.Add(n[idx_2] >= n[idx_1] + 1).OnlyEnforceIf(x[idx_1, idx_2])

# 线路覆盖约束
for line_name, edges in line_routes.items():
    line_edge_vars = []
    for edge in edges:
        o_, d_ = edge.split(",")
        o_idx, d_idx = station_name_to_idx[o_], station_name_to_idx[d_]
        line_edge_vars.append(x[o_idx, d_idx])
        line_edge_vars.append(x[d_idx, o_idx])
    
    # Gurobi: ptn.addConstr(grb.quicksum(...) >= 1)
    model.Add(sum(line_edge_vars) >= 1)

# 特殊线路覆盖约束 (Line 3/4 和 2/10)
# 处理 Line 3 + Line 4 的情况
edges_3_4 = list(set(line_routes["Line 3"] + line_routes["Line 4"]))
line_3_4_vars = []
for edge in edges_3_4:
    o_, d_ = edge.split(",")
    o_idx, d_idx = station_name_to_idx[o_], station_name_to_idx[d_]
    line_3_4_vars.append(x[o_idx, d_idx])
    line_3_4_vars.append(x[d_idx, o_idx])
# Gurobi: ptn.addConstr(grb.quicksum(...) >= 2)
model.Add(sum(line_3_4_vars) >= 2)


# 处理 Line 2 + Line 10 的情况
edges_2_10 = list(set(line_routes["Line 2"] + line_routes["Line 10"]))
line_2_10_vars = []
for edge in edges_2_10:
    o_, d_ = edge.split(",")
    o_idx, d_idx = station_name_to_idx[o_], station_name_to_idx[d_]
    line_2_10_vars.append(x[o_idx, d_idx])
    line_2_10_vars.append(x[d_idx, o_idx])
# Gurobi: ptn.addConstr(grb.quicksum(...) >= 2)
model.Add(sum(line_2_10_vars) >= 2)


# 5. 目标函数定义 (OR-Tools)
# =================================
objective_terms = []
for station, connections in metro_network.items():
    idx_1 = station_name_to_idx[station]
    for next_station, dist in connections.items():
        idx_2 = station_name_to_idx[next_station]
        # x 变量在 Gurobi 中直接与目标系数关联
        # 在 OR-Tools 中，我们创建一个表达式
        objective_terms.append(x[idx_1, idx_2] * dist)

model.Minimize(sum(objective_terms))
# model.Maximize(sum(objective_terms))


# 6. 求解与结果输出 (OR-Tools)
# =================================
solver = cp_model.CpSolver()
# 可以设置求解时间限制，例如：solver.parameters.max_time_in_seconds = 300.0
status = solver.Solve(model)

if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    print(f'BEST VALUE: {solver.ObjectiveValue()}')
    print('--- Path Found ---')
    for (i, j), var in x.items():
        if solver.Value(var) > 0:
            print(f"{idx_to_station_name[i]},{idx_to_station_name[j]}")
    print("-" * 20)
    print(f"Solver status: {solver.StatusName(status)}")
    print(f"Wall time: {solver.WallTime()}s")
else:
    print('No solution found.')

BEST VALUE: 65877.0
--- Path Found ---
莘庄,外环路
外环路,莲花路
莲花路,锦江乐园
锦江乐园,上海南站
上海南站,桂林公园
漕宝路,上海体育馆
上海体育馆,上海体育场
徐家汇,宜山路
陕西南路,上海图书馆
一大会址·黄陂南路,陕西南路
虹桥火车站,国家会展中心L17
虹桥2号航站楼,虹桥火车站
陆家嘴,豫园
浦东南路,陆家嘴
世纪大道,浦东南路
龙阳路,迎春路
宜山路,虹桥路
虹桥路,宋园路
东安路,龙华中路
上海体育场,东安路
春申路,莘庄
民生路,源深体育中心
源深体育中心,世纪大道
东明路,华鹏路
龙华中路,后滩
后滩,长清路
长清路,耀华路
耀华路,成山路
大世界,一大会址·黄陂南路
成山路,东明路
杨高中路,民生路
龙溪路,上海动物园
虹桥1号航站楼,虹桥2号航站楼
上海动物园,虹桥1号航站楼
水城路,龙溪路
伊犁路,水城路
宋园路,伊犁路
交通大学,徐家汇
上海图书馆,交通大学
豫园,大世界
桂林公园,漕宝路
华鹏路,下南路
下南路,北蔡
北蔡,陈春路
陈春路,莲溪路
莲溪路,华夏中路
华夏中路,龙阳路
迎春路,杨高中路
Dummy,春申路
国家会展中心L17,Dummy
--------------------
Solver status: OPTIMAL
Wall time: 146.610394s


## ortools for Q2

In [103]:
from ortools.sat.python import cp_model
import collections
# 2. 模型初始化 (OR-Tools)
# =================================
model = cp_model.CpModel()

# 添加虚拟节点 Dummy，与Gurobi代码逻辑一致
idx_to_station_name[0] = "Dummy,line0"
station_name_to_idx["Dummy,line0"] = 0

N = len(station_name_to_idx.keys())
M = 1000  # "Big M" 常数，与Gurobi代码一致

# 3. 变量定义 (OR-Tools)
# =================================
x = {}
y = {}
n = {}
out_flow = collections.defaultdict(list)

# 定义 y 和 n 变量
# 索引从1开始，因为虚拟节点0没有 y 和 n 变量
for i in range(1, N):
    y[i] = model.NewBoolVar(f"y_{idx_to_station_name[i]}")
    # n[i] 的上界可以是 N 或者 M，这里用 M 保持一致
    n[i] = model.NewIntVar(0, N + 1, f"n_{idx_to_station_name[i]}")

# 定义 x 变量，并构建出度邻接表 out_flow
# 真实站点之间的边
for station, connections in metro_network.items():
    idx_1 = station_name_to_idx[station]
    for next_station, dist in connections.items():
        idx_2 = station_name_to_idx[next_station]
        # 确保只为每个无向边创建一次变量（尽管模型处理有向边）
        if (idx_1, idx_2) not in x:
            x[idx_1, idx_2] = model.NewBoolVar(f"x_{station}_{next_station}")
        out_flow[idx_1].append(idx_2)

# 处理换乘站点的情况
for station, tgt_lines in trans_stations.items():
    if len(tgt_lines) == 1:
        continue 
    else:
        l = len(tgt_lines)
        stat_lines = list(tgt_lines)
        for i in range(l - 1):
            for j in range(i + 1, l):
                station1, station2 = f"{station},{stat_lines[i]}", f"{station},{stat_lines[j]}"
                idx_1, idx_2 = station_name_to_idx[station1], station_name_to_idx[station2]
                if (idx_1, idx_2) not in x:
                    x[idx_1, idx_2] = model.NewBoolVar(f"x_{station1}_{station2}")
                if (idx_2, idx_1) not in x: 
                    x[idx_2, idx_1] = model.NewBoolVar(f"x_{station2}_{station1}")
                if idx_2 not in out_flow[idx_1]: 
                    out_flow[idx_1].append(idx_2)
                if idx_1 not in out_flow[idx_2]: 
                    out_flow[idx_2].append(idx_1)
            


# 与虚拟节点相关的边
for i in range(1, N):
    x[0, i] = model.NewBoolVar(f"x_Dummy_{idx_to_station_name[i]}")
    x[i, 0] = model.NewBoolVar(f"x_{idx_to_station_name[i]}_Dummy")

# 4. 约束定义 (OR-Tools)
# =================================

# 流量守恒约束
for k in range(1, N):
    # 构建进入和流出k的完整边集，包括与虚拟节点的连接
    out_neighbors = out_flow[k] + [0]
    in_neighbors = out_flow[k] + [0] # 对于无向图的对称创建，in-neighbors等同于out-neighbors
    
    # 出度约束
    model.Add(sum(x.get((k, j), 0) for j in out_neighbors) == y[k])
    # 入度约束
    model.Add(sum(x.get((i, k), 0) for i in in_neighbors) == y[k])

# 虚拟节点流量平衡约束
model.Add(sum(x[0, i] for i in range(1, N)) == 1)
model.Add(sum(x[i, 0] for i in range(1, N)) == 1)


# 子回路消除 (MTZ) 约束
for i in range(1, N):
    # Gurobi: ptn.addConstr(n[i] <= M * y[i])
    # OR-Tools等价实现: 如果 y[i]为0, 那么 n[i]必须为0
    model.Add(n[i] == 0).OnlyEnforceIf(y[i].Not())
    # 或者直接使用线性约束，与 Gurobi 完全一致
    # model.Add(n[i] <= M * y[i]) # 这句也可以，但上面的逻辑更紧凑

# Gurobi: ptn.addConstr(n[idx_2] - n[idx_1] >= 1 + M * (x[idx_1, idx_2] - 1))
# OR-Tools 等价实现: 如果 x[idx_1, idx_2] = 1, 则 n[idx_2] >= n[idx_1] + 1
for station, connections in metro_network.items():
    idx_1 = station_name_to_idx[station]
    for next_station, dist in connections.items():
        idx_2 = station_name_to_idx[next_station]
        # 添加两个方向的约束
        model.Add(n[idx_2] >= n[idx_1] + 1).OnlyEnforceIf(x[idx_1, idx_2])
# 换乘站的 n 
for station, tgt_lines in trans_stations.items():
    if len(tgt_lines) == 1:
        continue 
    else:
        l = len(tgt_lines)
        stat_lines = list(tgt_lines)
        for i in range(l - 1):
            for j in range(i + 1, l):
                station1, station2 = f"{station},{stat_lines[i]}", f"{station},{stat_lines[j]}"
                idx_1, idx_2 = station_name_to_idx[station1], station_name_to_idx[station2]
                model.Add(n[idx_2] >= n[idx_1] + 1).OnlyEnforceIf(x[idx_1, idx_2])
                model.Add(n[idx_1] >= n[idx_2] + 1).OnlyEnforceIf(x[idx_2, idx_1])


# 线路覆盖约束
# for line_name, edges in line_routes.items():
#     line_edge_vars = []
#     for edge in edges:
#         o_, d_ = edge.split(",")
#         o_idx, d_idx = station_name_to_idx[o_], station_name_to_idx[d_]
#         line_edge_vars.append(x[o_idx, d_idx])
#         line_edge_vars.append(x[d_idx, o_idx])
    
#     # Gurobi: ptn.addConstr(grb.quicksum(...) >= 1)
#     model.Add(sum(line_edge_vars) >= 1)

# # 特殊线路覆盖约束 (Line 3/4 和 2/10)
# # 处理 Line 3 + Line 4 的情况
# edges_3_4 = list(set(line_routes["Line 3"] + line_routes["Line 4"]))
# line_3_4_vars = []
# for edge in edges_3_4:
#     o_, d_ = edge.split(",")
#     o_idx, d_idx = station_name_to_idx[o_], station_name_to_idx[d_]
#     line_3_4_vars.append(x[o_idx, d_idx])
#     line_3_4_vars.append(x[d_idx, o_idx])
# # Gurobi: ptn.addConstr(grb.quicksum(...) >= 2)
# model.Add(sum(line_3_4_vars) >= 2)


# # 处理 Line 2 + Line 10 的情况
# edges_2_10 = list(set(line_routes["Line 2"] + line_routes["Line 10"]))
# line_2_10_vars = []
# for edge in edges_2_10:
#     o_, d_ = edge.split(",")
#     o_idx, d_idx = station_name_to_idx[o_], station_name_to_idx[d_]
#     line_2_10_vars.append(x[o_idx, d_idx])
#     line_2_10_vars.append(x[d_idx, o_idx])
# # Gurobi: ptn.addConstr(grb.quicksum(...) >= 2)
# model.Add(sum(line_2_10_vars) >= 2)

# 5. 目标函数定义 (OR-Tools)
# =================================
objective_terms = []
for station, connections in metro_network.items():
    idx_1 = station_name_to_idx[station]
    for next_station, dist in connections.items():
        idx_2 = station_name_to_idx[next_station]
        # x 变量在 Gurobi 中直接与目标系数关联
        # 在 OR-Tools 中，我们创建一个表达式
        objective_terms.append(x[idx_1, idx_2] * dist)

# 5.1 换乘站的权重设置为 0 
for station, tgt_lines in trans_stations.items():
    if len(tgt_lines) == 1:
        continue 
    else:
        l = len(tgt_lines)
        stat_lines = list(tgt_lines)
        for i in range(l - 1):
            station1, station2 = f"{station},{stat_lines[i]}", f"{station},{stat_lines[i + 1]}"
            idx_1, idx_2 = station_name_to_idx[station1], station_name_to_idx[station2]
            objective_terms.append(x[idx_1, idx_2] * 0)
            objective_terms.append(x[idx_2, idx_1] * 0)
            

# model.Minimize(sum(objective_terms))
model.Maximize(sum(objective_terms))


# 6. 求解与结果输出 (OR-Tools)
# =================================
solver = cp_model.CpSolver()
# 可以设置求解时间限制，例如：solver.parameters.max_time_in_seconds = 300.0

model_proto = model.Proto()

# 4. 打印变量和约束的数量
num_variables = len(model_proto.variables)
num_constraints = len(model_proto.constraints)

print(f"模型中的变量数量: {num_variables}")
print(f"模型中的约束数量: {num_constraints}")

status = solver.Solve(model)

result_list = []

if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    print(f'BEST VALUE: {solver.ObjectiveValue()}')
    print('--- Path Found ---')
    for (i, j), var in x.items():
        if solver.Value(var) > 0:
            result_list.append(f"{idx_to_station_name[i]},{idx_to_station_name[j]}")
            # print(f"{idx_to_station_name[i]},{idx_to_station_name[j]}")
            # print(solver.Value(n[i]), solver.Value(n[j]))
    print("-" * 20)
    print(f"Solver status: {solver.StatusName(status)}")
    print(f"Wall time: {solver.WallTime()}s")
else:
    print('No solution found.')

# Shanghai
# 模型中的变量数量: 3298
# 模型中的约束数量: 2788

# Beijing
# 模型中的变量数量: 3138
# 模型中的约束数量: 2647

# Nanjing
# 模型中的变量数量: 1516
# 模型中的约束数量: 1271

模型中的变量数量: 3298
模型中的约束数量: 2788
BEST VALUE: 475152.0
--- Path Found ---
--------------------
Solver status: OPTIMAL
Wall time: 82.934244s


## Reconstruct from result

In [104]:
import datetime
from collections import defaultdict

# 核心排序逻辑 (与之前版本相同，无需修改)
def order_path_with_transfers(raw_segments):
    """将包含换乘的无序路径段列表整理成有序路径。"""
    path_graph = {}
    valid_segments_count = 0
    for segment in raw_segments:
        parts = [p.strip() for p in segment.split(',')]
        if len(parts) != 4:
            continue
        
        start_station, start_line, end_station, end_line = parts
        start_node = (start_station, start_line)
        end_node = (end_station, end_line)
        
        if start_node in path_graph:
            raise ValueError(f"数据错误: 节点 {start_node} 有多个出路。")
            
        path_graph[start_node] = end_node
        valid_segments_count += 1
        
    try:
        start_node = next(key for key in path_graph if key[0] == 'Dummy')
    except StopIteration:
        raise ValueError("路径错误: 在数据中找不到 'Dummy' 作为起点。")

    ordered_path = []
    current_node = start_node
    visited_nodes = set()
    # print(valid_segments_count)
    for _ in range(valid_segments_count):
        if current_node in visited_nodes:
            raise ValueError(f"路径中存在环或断裂: 节点 {current_node} 被重复访问。")
        visited_nodes.add(current_node)
        
        if current_node not in path_graph:
            raise ValueError(f"路径断裂: 找不到从节点 {current_node} 出发的路段。")
            
        next_node = path_graph[current_node]
        ordered_path.append((current_node, next_node))
        print(current_node, next_node)
        current_node = next_node
        if current_node[0] == 'Dummy':
            break

    if len(ordered_path) != valid_segments_count:
        print(f"警告: 生成的有序路径长度 ({len(ordered_path)}) 与有效数据段数量 ({valid_segments_count}) 不匹配。")
        
    return ordered_path

def reconstruct_save_and_display_final(raw_route_data, metro_network, city = "BJ"):
    """
    主函数：适配新的 metro_network 格式，解析、排序、计算距离，并将结果显示和保存。

    Args:
        raw_route_data (list): "起点,起点线路,终点,终点线路" 格式的字符串列表。
        metro_network (defaultdict): 站点线路对之间的距离查找字典。
                                     格式: {(st1, ln1): {(st2, ln2): dist}, ...}
    """
    output_lines = []
    
    def log(message):
        """辅助函数，同时添加到输出列表并打印到控制台。"""
        print(message)
        output_lines.append(message)

    # log("--- 步骤 1: 扫描原始数据 ---")
    valid_data = []
    for item in raw_route_data:
        if len(item.split(',')) != 4:
            log(f"忽略格式错误的行: '{item}'")
        else:
            valid_data.append(item)
    
    if not valid_data:
        log("没有解析到任何有效数据。")
        return

    # log("\n--- 步骤 2: 对路径段进行排序 (包含换乘逻辑) ---")
    try:
        ordered_path = order_path_with_transfers(valid_data)
    except (ValueError, KeyError) as e:
        log(f"排序过程中出错: {e}")
        return
    
    # 将 valid_data 转换成节点对的集合，用于后续比较
    all_valid_nodes = set()
    for item in valid_data:
        parts = [p.strip() for p in item.split(',')]
        start_node = (parts[0], parts[1])
        end_node = (parts[2], parts[3])
        all_valid_nodes.add((start_node, end_node))

    try:
        ordered_path = order_path_with_transfers(valid_data)
    except (ValueError, KeyError) as e:
        log(f"排序过程中出错: {e}")
        return

    # --- 增加调试信息 ---
    if len(ordered_path) != len(all_valid_nodes):
        used_nodes = set(ordered_path)
        isolated_nodes = all_valid_nodes - used_nodes
        
        log(f"\n🚨 警告: 路径不完整或包含孤立段！")
        log(f"   - 总有效段数: {len(all_valid_nodes)}")
        log(f"   - 主路径段数: {len(used_nodes)}")
        log(f"   - 未使用的孤立段 ({len(isolated_nodes)}):")
        for start_n, end_n in isolated_nodes:
            log(f"     -  {start_n} -> {end_n}")

    # log("\n--- 步骤 3: 计算距离并生成最终有序路线 ---")
    log("\n================== 复原的完整路线 ==================")
    
    total_distance = 0.0
    
    for start_node, end_node in ordered_path:
        start_station, start_line = start_node
        end_station, end_line = end_node

        if "Dummy" in (start_station, end_station):
            continue
        
        segment_distance = 0.0
        
        if start_station == end_station: # 换乘
            segment_distance = 0.0  # 换乘距离为0
            total_distance += segment_distance
            log(f"【换乘】在 [{start_station}] 站, 从 <{start_line}> 换乘至 <{end_line}> (距离: {segment_distance})")
        else: # 乘车
            # 直接使用 (站点, 线路) 元组作为键进行查询
            start_node_d = f"{start_node[0]},{start_node[1]}"
            end_node_d = f"{end_node[0]},{end_node[1]}"
            dist = metro_network.get(start_node_d, {}).get(end_node_d)
            
            # 如果是 defaultdict，它不会返回 None，而是空字典，所以 get 更好
            # 如果找不到，尝试反向查询
            if dist is None:
                dist = metro_network.get(end_node_d, {}).get(start_node_d)

            if dist is not None:
                segment_distance = dist
                total_distance += segment_distance
                log(f"【乘车】从 [{start_station}] 到 [{end_station}], 乘坐线路: <{start_line}> (距离: {segment_distance})")
            else:
                log(f"【乘车】从 [{start_station}] 到 [{end_station}], 乘坐线路: <{start_line}> (警告: 未在 metro_network 中找到距离信息)")

    log("==================================================")
    log(f"\n总路线长度: {total_distance}")
    
    # --- 步骤 4: 保存到文件 ---
    timestamp = datetime.datetime.now().strftime("%Y%m%d-%H_%M_%S")
    filename = f"./data/PublicTransit/ROUTE_{city}_{timestamp}.txt"
    
    try:
        with open(filename, 'w', encoding='utf-8') as f:
            f.write("\n".join(output_lines))
        print(f"\n✅ 结果已成功保存到文件: {filename}")
    except IOError as e:
        print(f"\n❌ 保存文件失败: {e}")

# ========================== 示例使用 ==========================


reconstruct_save_and_display_final(result_list, metro_network, city)

('Dummy', 'line0') ('滴水湖', 'Line 16')
('滴水湖', 'Line 16') ('临港大道', 'Line 16')
('临港大道', 'Line 16') ('书院', 'Line 16')
('书院', 'Line 16') ('惠南东', 'Line 16')
('惠南东', 'Line 16') ('惠南', 'Line 16')
('惠南', 'Line 16') ('野生动物园', 'Line 16')
('野生动物园', 'Line 16') ('新场', 'Line 16')
('新场', 'Line 16') ('航头东', 'Line 16')
('航头东', 'Line 16') ('鹤沙航城', 'Line 16')
('鹤沙航城', 'Line 16') ('周浦东', 'Line 16')
('周浦东', 'Line 16') ('罗山路', 'Line 16')
('罗山路', 'Line 16') ('华夏中路', 'Line 16')
('华夏中路', 'Line 16') ('华夏中路', 'Line 13')
('华夏中路', 'Line 13') ('莲溪路', 'Line 13')
('莲溪路', 'Line 13') ('陈春路', 'Line 13')
('陈春路', 'Line 13') ('北蔡', 'Line 13')
('北蔡', 'Line 13') ('下南路', 'Line 13')
('下南路', 'Line 13') ('华鹏路', 'Line 13')
('华鹏路', 'Line 13') ('东明路', 'Line 13')
('东明路', 'Line 13') ('成山路', 'Line 13')
('成山路', 'Line 13') ('长清路', 'Line 13')
('长清路', 'Line 13') ('世博大道', 'Line 13')
('世博大道', 'Line 13') ('世博会博物馆', 'Line 13')
('世博会博物馆', 'Line 13') ('马当路', 'Line 13')
('马当路', 'Line 13') ('一大会址·新天地', 'Line 13')
('一大会址·新天地', 'Line 13') ('淮海中路', 

In [41]:
chck = set(result_list)
len(chck)

345

## 测试

In [ ]:
# 【所需边数最小的（44条边）】
# 国家会展中心L17,虹桥火车站,2523
# 虹桥火车站,虹桥2号航站楼,696
# 虹桥2号航站楼,淞虹路,5921
# 淞虹路,北新泾,1367
# 北新泾,威宁路,1260
# 威宁路,娄山关路,1661
# 娄山关路,长风公园,1487
# 长风公园,大渡河路,784
# 大渡河路,金沙江路,1676
# 金沙江路,曹杨路,903
# 曹杨路,武宁,1050
# 武宁,武定路,970
# 武定路,静安寺,1030
# 静安寺,常熟路,1091
# 常熟路,陕西南路,939
# 陕西南路,一大会址·新天地,1591
# 一大会址·新天地,老西门,880
# 老西门,陆家浜路,825
# 陆家浜路,小南门,1475
# 小南门,商城路,2398
# 商城路,世纪大道,979
# 世纪大道,源深体育中心,890
# 源深体育中心,民生路,907
# 民生路,杨高中路,928
# 杨高中路,迎春路,633
# 迎春路,龙阳路,2275
# 龙阳路,华夏中路,4355
# 华夏中路,莲溪路,1589
# 莲溪路,御桥,1342
# 御桥,浦三路,3221
# 浦三路,三林东,1613
# 三林东,三林,1126
# 三林,东方体育中心,3516
# 东方体育中心,龙耀路,2164
# 龙耀路,云锦路,934
# 云锦路,龙华,945
# 龙华,龙漕路,1124
# 龙漕路,石龙路,1484
# 石龙路,上海南站,1314
# 上海南站,锦江乐园,2089
# 锦江乐园,莲花路,1633
# 莲花路,外环路,1459
# 外环路,莘庄,1303
# 莘庄,春申路,1588
# 69938 



| 序号 | 线路 | 起点 | 终点| 距离 | 
|:---:| :---: |:----: | :---:| :---: |
| 1 | L17 | 国家会展中心L17 | 虹桥火车站 | 2523 |
| 2 | L2 | 虹桥火车站 | 虹桥2号航站楼 | 696 | 
| 3 | L10 | 虹桥2号航站楼 | 虹桥1号航站楼 | 2026 |
|4 | L10  | 虹桥1号航站楼 | 上海动物园 | 2016 |
| 5 | L10 | 上海动物园 | 龙溪路 | 1244| 
|6 | L10| 龙溪路 | 水城路 | 1303 |
| 7| L10| 水城路 | 伊犁路 | 1167 |
| 8| L10 | 伊犁路 | 宋园路 | 800 |
| 9| L10| 宋园路 | 虹桥路 | 1053 |
| 10 | L3 | 虹桥路 | 宜山路 | 1323 |
| 11| L9 | 宜山路 | 徐家汇 | 1505 |
| 12| L11 | 徐家汇 | 交通大学 | 905 |
| 13| L11 | 交通大学 | 上海图书馆 | 1145 |
| 14| L11 | 上海图书馆 | 陕西南路 | 1618 |
| 15 | L1 | 陕西南路 | 一大会址·黄陂南路 | 1407 |
|16 |  L14 | 一大会址·黄陂南路 | 大世界 | 710 | 
| 17| L14 | 大世界 | 豫园 | 740 |
|8| L14 | 豫园 | 陆家嘴 | 1600 |
|9| L14 | 陆家嘴 | 浦东南路 | 1010 |
|20| L2 | 浦东南路 | 世纪大道 | 1245 |
|21| L6 | 世纪大道 | 源深体育中心 | 890 |
|22| L6| 源深体育中心 | 民生路 | 907 |
|23| L18 | 民生路 | 杨高中路 | 928 |
|24| L18| 杨高中路 | 迎春路 | 633 |
|25| L18 | 迎春路 | 龙阳路 | 2275 |
|26| L16 | 龙阳路 | 华夏中路 | 4355 |
|27| L13 | 华夏中路 | 莲溪路 | 1589 |
|28| L13| 莲溪路 | 陈春路 | 1239 |
|29 |L13 | 陈春路 | 北蔡 | 980 |
|30 | L13| 北蔡 | 下南路 | 1128 |
|31|L13 | 下南路 | 华鹏路 | 1361 |
|32| L13| 华鹏路 | 东明路 | 1464 |
|33 | L13| 东明路 | 成山路 | 1397 |
|34 |L8 | 成山路 | 耀华路 | 908 |
|35| L7 | 耀华路 | 长清路 | 926 |
|36| L7| 长清路 | 后滩 | 1263 |
|37| L7| 后滩 | 龙华中路 | 2247 |
|38| L7| 龙华中路 | 东安路 | 728 |
|39| L4| 东安路 | 上海体育场 | 1220 |
|40 | L4| 上海体育场 | 上海体育馆 | 685 |
|41 |L1 | 上海体育馆 | 漕宝路 | 1534 |
|42 |L12 | 漕宝路 | 桂林公园 | 1284 |
|43 |L15 | 桂林公园 | 上海南站 | 1828 |
|44 |L1 | 上海南站 | 锦江乐园 | 2089 |
|45 |L1 | 锦江乐园 | 莲花路 | 1633 |
|46 |L1 | 莲花路 | 外环路 | 1459 |
|47 |L1 | 外环路 | 莘庄 | 1303 |
|48 |L5 | 莘庄 | 春申路 | 1588 |
